In [9]:
import pandas as pd
import json
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
arquivo = open("C:\Users\1234\Desktop\Insper\2° Semestre\Ciência dos dados\GitHub CD\CD\Projeto2\\export.json")
linhas = arquivo.readlines()
texto = "".join(linhas)
dados_fire = json.loads(texto)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-12-299a006b362a>, line 1)

In [9]:
for k,v in dados_fire["logs"].items():
    # print(k,v) # habilite para imprimir as chaves de mais alto nível
    for chave, valor in v.items():
        # print(chave, valor) # habilite para imprimir as chaves inferiores
        pass # desabilite quando reabilitar a linha de cima

NameError: name 'dados_fire' is not defined

In [10]:
lista_dicionarios = []

for k,v in dados_fire["logs"].items():
    lista_dicionarios.append(v)

NameError: name 'dados_fire' is not defined

In [11]:
dataframe = pd.DataFrame.from_records(lista_dicionarios)

NameError: name 'pd' is not defined

In [14]:
dataframe

tempo    tipo     user_id versao
0    1479324005419  inicio  2485808503      B
1    1479323869493  inicio  1328592160      A
2    1479330035958   final  3774183163      B
3    1479326509406   final  1069924001      B
4    1479322263526  inicio  3182908516      B
5    1479325011188  inicio   107073801      B
6    1479322709781   final  3158981088      A
7    1479322745195  inicio  2784224332      B
8    1479322468018  inicio  3848348405      A
9    1479327799457  inicio   539025276      B
10   1479329094750  inicio  1199188587      A
11   1479332037361   final  3081684687      A
12   1479330719874  inicio  3332280950      A
13   1479324126064   final  3839108837      A
14   1479323072672  inicio  2800236100      B
15   1479341685469  inicio  1010704001      A
16   1479327854516  inicio  4283386924      A
17   1479325518155  inicio   572274542      B
18   1479335032628  inicio  2200140917      A
19   1479343100709   final  3088059985      A
20   1479322765615  inicio  1367474880      A
21   1479323768088  inicio  3211109805      B
22   1479330028383  inicio  3774183163      B
23   1479335213423  inicio  4265449876      A
24   1479330742229   final  3332280950      A
25   1479329019757   final  1110907384      A
26   1479326728362   final   313146214      B
27   1479327569703  inicio  3476142167      A
28   1479341697117   final  1010704001      A
29   1479322555843   final  2527838409      A
..             ...     ...         ...    ...
145  1479327826202  inicio  1386160978      A
146  1479325797386  inicio  1900471385      B
147  1479326527125   final  3303704572      A
148  1479330878095   final   313146214      B
149  1479324041808   final   201603664      B
150  1479335346395   final  4265449876      A
151  1479324037966  inicio   201603664      B
152  1479321858736   final  1386160978      B
153  1479325382240  inicio   858371356      B
154  1479323853923  inicio  3211109805      B
155  1479323126273  inicio  1386160978      B
156  1479333718874  inicio  2725705737      B
157  1479333476747  inicio  2627218697      B
158  1479333638626  inicio  2725705737      B
159  1479326511858  inicio    75772886      A
160  1479325806539   final  1379609113      B
161  1479326714667  inicio   313146214      B
162  1479323560906  inicio  2086948951      A
163  1479332881161  inicio     6930556      B
164  1479325338977  inicio   969870491      B
165  1479322257609  inicio   612948909      B
166  1479325130433  inicio  3442786406      B
167  1479323249546  inicio   265861560      A
168  1479328338694   final  4283386924      A
169  1479333333346  inicio  4264934178      A
170  1479323900274   final  1919271831      A
171  1479325449554  inicio   572274542      B
172  1479322808424   final  1367474880      A
173  1479325143071  inicio  3442786406      A
174  1479343040256  inicio  1109903459      B

[175 rows x 4 columns]

## Convertendo para um DataFrame de delta_t

In [15]:
agrupado = dataframe.groupby(by=["user_id", "versao"])

In [16]:
users = set(dataframe.user_id) # We convert it to a set so that each user appear only once

In [17]:
users = list(users)

In [18]:
resultados = []

for u in users:
    for versao in ["A", "B"]:
        if (u, versao) in agrupado.groups.keys(): # Check if a specific pair user/type of test happened
            grupo1 =  agrupado.get_group((u, versao))
            grupo1 = grupo1.sort_values(by="tempo") # Para evitar que inícios sem fim gerem tempos negativos
            
            if len(grupo1[grupo1.tipo=="final"])>= 1 and len(grupo1[grupo1.tipo=="inicio"]) >= 1:
                delta_t = grupo1[grupo1.tipo=="final"].iloc[-1].tempo - grupo1[grupo1.tipo=="inicio"].iloc[0].tempo
                resultados.append({"user_id":u, "delta_t":delta_t, "versao":versao})

In [19]:
grupo1.sort_values(by="tempo")

tempo   tipo     user_id versao
147  1479326527125  final  3303704572      A

In [20]:
dataframe_resumo = pd.DataFrame.from_records(resultados)

In [21]:
dataframe_resumo.delta_t/=1000 # Convertendo milissegundos em segundos

In [22]:
dataframe_resumo

delta_t     user_id versao
0     484.178  4283386924      A
1       4.516  3185562697      B
2       3.842   201603664      B
3     120.949  2086948951      A
4      16.300  1900471385      B
5    6830.549  1199188587      A
6      22.355  3332280950      A
7       8.401  2974018681      B
8     218.225  1010704001      A
9      26.417  2693455507      B
10     42.809  1367474880      A
11     17.212  2527838409      A
12     20.074  3839108837      A
13    330.177  3848348405      A
14   -139.168  3848348405      B
15      7.575  3774183163      B
16     27.710  3541680390      B
17    120.111  2627218697      A
18     47.050  4214608654      A
19     13.525  1386160978      A
20   1323.799  1386160978      B
21  11743.062   313146214      B
22    132.972  4265449876      A
23    147.886  1919271831      A
24    175.650  3211109805      B
25     13.742  1110907384      A